In [ ]:
import numpy as np
import pandas as pd
import pickle
import scipy

##from statannot import add_stat_annotation
from statannotations.Annotator import Annotator
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
all_corr_sep_sequences = np.load('./all_corr_sep_sequences.npy',allow_pickle=True)

In [ ]:
tot_iter_run = 100
all_below_thresh = []
for seq in range(4):
    all_corr_sep = all_corr_sep_sequences[seq]
    below_thresh= tot_iter_run*np.ones((len(all_corr_sep), 26*4))
    thresh_cross = 0.2
    for rep in range(len(all_corr_sep)):
        print(seq, rep)
        corr_sep_curr = all_corr_sep[rep]
        for i in range(np.shape(corr_sep_curr)[1]):
            
            plt.plot(corr_sep_curr[:,i], label = str(i))
            if len(np.where(corr_sep_curr[:,i]<thresh_cross)[0])>0:
                below_thresh[rep, i] = np.where(corr_sep_curr[:,i]<thresh_cross)[0][0]
                print(np.where(corr_sep_curr[:,i]<thresh_cross)[0][0])
        plt.legend()
        plt.show()
    all_below_thresh.append(below_thresh)

In [ ]:
n_iter =10
all_titles= ['visual then same reward','visual then different reward', 'same reward then visual','different reward then visual' ]
##fig, axs = plt.subplots(1,4,figsize = (23,6))
##fig, axs = plt.subplots(1,4,figsize = (18,3))
for curr_fig in range(0, 4):
    all_loc = []
    when_cross = []
    below_thresh = np.column_stack((all_below_thresh[curr_fig][:,1], all_below_thresh[curr_fig][:,6]))##all_below_thresh[curr_fig][:,1]
    cross1 = below_thresh[:,0]<tot_iter_run
    cross2 = below_thresh[:,1]<tot_iter_run
    learned = cross1*cross2

    for i in range(len(below_thresh)):
        for j in range(len(below_thresh[0])):
            if learned[i]== True:  ## only plotting simulations that learn fully
                when_cross.append(below_thresh[i,j]*n_iter)
                all_loc.append(j)
    if curr_fig == 0:
        Xplot = pd.DataFrame(np.array([when_cross, all_loc, [curr_fig]*len(when_cross)]).T)
    else:
        Xplot = pd.concat([Xplot,pd.DataFrame(np.array([when_cross, all_loc, [curr_fig]*len(when_cross)]).T)],ignore_index=True)


In [ ]:
all_below_thresh = np.array(all_below_thresh)
fig, axs = plt.subplots(1, figsize = (14,8))

##sb.boxplot(x = 2, y = 0, data =Xplot,hue = 1,palette =[ 'lightskyblue','#5B6BA8' ], ax = axs, showfliers= False)
sb.boxplot(x = 2, y = 0, data =Xplot,hue = 1,palette =[ '#5B6BA8','lightskyblue' ], ax = axs, showfliers= False)
pairs=[((0.0,0.0),(0.0,1.0)), ((1.0,0.0),(1.0,1.0)), ((2.0,0.0),(2.0,1.0)), ((3.0,0.0),(3.0,1.0))]

annot = Annotator(axs, pairs, data=Xplot, x=2, y=0, hue = 1)
##Annotator.new_plot(ax= axs, pairs, data=Xplot, x=2, y=0, hue = 1)
annot.configure(test='t-test_paired').apply_test().annotate()
##annot.configure(test='Wilcoxon').apply_test().annotate()

xloc = -0.24
all_below_thresh_jitter = (all_below_thresh*n_iter) + np.random.normal(loc=0, scale=0.05, size=np.shape(all_below_thresh))
for seq in range(0,4):
    for i in range(0,20):
        jitterx1 = np.random.normal(loc=0, scale=0.05)
        jitterx2 = np.random.normal(loc=0, scale=0.05)
        if np.max(all_below_thresh_jitter[seq][i,0:3]) < tot_iter_run*n_iter-1 and np.max(all_below_thresh_jitter[seq][i,5:8]) < tot_iter_run*n_iter-1:
            ##print(all_below_thresh_jitter[seq][i,0:3], all_below_thresh_jitter[seq][i,5:8])
            axs.plot([xloc + jitterx1,xloc+0.5 + jitterx2],[np.mean(all_below_thresh_jitter[seq][i,0:3]), np.mean(all_below_thresh_jitter[seq][i,5:8])], linewidth=0.5, color = 'grey', alpha = 0.5, rasterized = False)
            axs.scatter([xloc + jitterx1,xloc+0.5 + jitterx2],[np.mean(all_below_thresh_jitter[seq][i,0:3]), np.mean(all_below_thresh_jitter[seq][i,5:8])], color = ['navy', 'dodgerblue'],s= 20)
    xloc= xloc + 1##0.99
    
axs.set_ylim(0,950)
plt.legend([])
axs.set_xticklabels(all_titles, rotation = 20)
axs.set_ylabel('Number of iterations to decorrelate')
plt.savefig('../FIGURES/decorrelation_time_diff_sequence_all_in_one_connected_p_val_t_test_paired.pdf', format='pdf', dpi=500)

In [ ]:
for seq in range(0, 4):
    r0 = np.array(Xplot.loc[(Xplot[2]==seq) &(Xplot[1]==0)][0])
    r1 = np.array(Xplot.loc[(Xplot[2]==seq) &(Xplot[1]==1)][0])
    tstat,pval = scipy.stats.ttest_rel(r0, r1)
    print(all_titles[seq], tstat, pval, len(r0),len(r1))